In [ ]:
__author__ = 'Chien-Hsiu Lee <chien-hsiu.lee@noirlab.edu>'
__version__ = '20210808' # yyyymmdd
__datasets__ = ['']
__keywords__ = ['ANTARES', 'TOMtoolkit']

# Interfacing ANTARES with TOMtoolkit via API

**This notebook requires the installation of ANTARES client (https://noao.gitlab.io/antares/client/) and TOMtoolkit ((https://tom-toolkit.readthedocs.io/en/latest/introduction/getting_started.html#installing-the-tom-toolkit-and-django). Once a TOMtoolkit webportal is launched, there are examples and formats of how to interact with TOMtoolkit via Django Restframe API at http://127.0.0.1:8000/api locally. In this example, however, we are using ANTARES TOM (tom.antares.noirlab.edu) as an instance, to demonstrate how to interact with TOMs set-up in public domains**



TOMtoolkit is an open source Target and Observation Manager (TOM) that allows users to coordinately annotate alerts, trigger observations, gather and share photometry and spectra, and plan subsequent follow-ups. Instead of going into each steps with the GUI manually, we can interact with TOMtoolkit in a programmatic manner, including creating targets from ANTARES, uploading photometry data (light curves) from ANTARES alert database, and more with the Django Restframe API as follows.


### Table of contents
* [Imports & setup](#import)
* [Create a TOMtoolkit target from an ANTARES locus object](#target)
* [Upload photometry/light curve from ANTARES alert database](#dataproduct)
* [Get spectra from TNS and upload to TOMtoolkit](#getspectra)


<a class="anchor" id="import"></a>
# Imports & Setup
The first step is to import ANTARES client, API function, and define API token for TOMtoolkit


In [5]:
#! pip install requests_toolbelt <-execute this line to install request_toolbelt for submitting multipart/form-data 
from requests_toolbelt import MultipartEncoder
import antares_client
#from antares_client.search import get_by_id, get_by_ztf_object_id, search
import astropy.time as atime
import datetime
import requests

#use http post https://tom.antares.noirlab.edu/api-token-auth/ username=YOUR_USERNAME password=YOUR_PASSWORD
#to get API token

#token can be generated by TOMtoolkit. See glossary for more detail
token = 'YOUR_TOKEN'
def api(method, endpoint, data=None):
    headers = {'Authorization': f'token {token}'}
    response = requests.request(method, endpoint, json=data, headers=headers)
    return response

<a class="anchor" id="target"></a>
# Create a TOMtoolkit target from an ANTARES locus object
Given a target ID (from ANTARES or ZTF), we can create a new target in the TOMtoolkit from an ANTARES locus object

In [6]:
#create a new target with API post
ztf_object_id="ZTF19aapreis"

locus = antares_client.search.get_by_ztf_object_id(ztf_object_id)
print(locus.properties['ztf_object_id'],locus.ra,locus.dec)

data = {
"targetextra_set":[],
"aliases":[],
"groups":[],
"name":ztf_object_id,
"type":"SIDEREAL",
"ra":locus.ra,
"dec":locus.dec,
"epoch":None,
"parallax":None,
"pm_ra":None,
"pm_dec":None,
"galactic_lng":None,
"galactic_lat":None,
"distance":None,
"distance_err":None,
"scheme":"",
"epoch_of_elements":None,
"mean_anomaly":None,
"arg_of_perihelion":None,
"eccentricity":None,
"lng_asc_node":None,
"inclination":None,
"mean_daily_motion":None,
"semimajor_axis":None,
"epoch_of_perihelion":None,
"ephemeris_period":None,
"ephemeris_period_err":None,
"ephemeris_epoch":None,
"ephemeris_epoch_err":None,
"perihdist":None
}



response = api('POST', 'https://tom.antares.noirlab.edu/api/targets/', data)

print(f'HTTP code: {response.status_code}, {response.reason}')
#if response.status_code in (200, 400):
#    print(f'JSON response: {response.json()}')




ZTF19aapreis 314.2623563407407 14.204495246296297
HTTP code: 201, Created


Once a target is created (or if it already exists in the target list), we can query the target ID in the TOM database.

In [14]:
#get target list in the TOMtoolkit and the id of the newly created target
response = api('GET', 'https://tom.antares.noirlab.edu/api/targets')

print(f'HTTP code: {response.status_code}, {response.reason}')
#if response.status_code in (200, 400):
#    print(f'JSON response: {response.json()}')

#this following piece of code lists targets in the TOM database, finds the ID for our newly ingested target, and stores that information in the target_id variable.
target_id = 0
target_list=response.json()
for i in target_list['results']:
    print(i['id'],i['name'])
    if i['name'] == ztf_object_id:
        target_id = i['id']

HTTP code: 200, OK
3 ZTF18abhjrcf
4 Test
5 TEst 2
6 ZTF18acszjan
7 ZTF18acrulmj
8 ZTF188abhjrcf
9 ZTF18abhjrcdef
10 ANT2020pykx2
11 ZTF20acklcyp
12 20xkx test
13 xox test
14 ZTF21abrpeix
15 ZTF18aboerzk
16 ZTF18acvdmdr-kostya
17 ZTF21abiatkr
18 ZTF21abpxuss
19 ZTF21abrpgcf
20 ZTF18acvdmdr-kostya-1
21 ZTF21abqvyqq
22 ZTF21absjqbd
23 ZTF19abffzmf
24 ZTF21abihsto
31 ZTF19aapreis


In [15]:
#generate ZTF light curve from ANTARES alert database; upload it to TOMtoolkit 
import pandas as pd
#obj_id="ANT2020pykx2"
#ztf_object_id="ZTF19aapreis"
#locus = antares_client.search.get_by_ztf_object_id(ztf_object_id)
print(locus.properties['ztf_object_id'])
df = locus.lightcurve
dd=df[df["ant_survey"]==1][["ant_mjd","ant_passband","ant_mag","ant_magerr"]]
new_row = pd.DataFrame({'ant_mjd':'time','ant_passband':'filter','ant_mag':'magnitude','ant_magerr':'error'}, index=[0])
dg = pd.concat([new_row, dd]).reset_index(drop = True)
dg.head(5)
dg.to_csv(str(locus.properties['ztf_object_id'])+'.csv', index = False, header = None)
#locus.lightcurve[["ant_survey==1"]][["ant_mjd","ant_passband","ant_mag","ant_magerr"]]

m = MultipartEncoder(
    fields={'target': str(target_id), 'data_product_type': 'photometry',
            'file': (str(locus.properties['ztf_object_id'])+'.csv', open(str(locus.properties['ztf_object_id'])+'.csv', 'rb'), 'text/csv')}
    )

#print(m)
r = requests.post('https://tom.antares.noirlab.edu/api/dataproducts/', data=m,
                  headers={'Content-Type': m.content_type, 'Authorization': f'token {token}'})

print(r.content)

ZTF19aapreis
b'{"id":28,"product_id":null,"target":31,"observation_record":null,"data":"http://tom.antares.noirlab.edu/data/ZTF19aapreis/none/ZTF19aapreis_jUPAhNj.csv","extra_data":"","data_product_type":"photometry","reduceddatum_set":[],"groups":[]}'


<a class="anchor" id="getspectra"></a>
# Get spectra from TNS and upload to TOMtoolkit
We can also search for TNS and retrieve corresponding spectra of the target. First we load query libraries from TNS and put in our credentials.

In [16]:
#get spectra from TNS
#First import the TNS API

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Avgust 2021

Developed and tested on:

- Linux 20.04 LTS
- Windows 10
- Python 3.8 (Spyder 4)

@author: Nikola Knezevic ASTRO DATA
"""

import os
import requests
import json
from collections import OrderedDict
import time

#----------------------------------------------------------------------------------

#TNS                 = "www.wis-tns.org"
TNS                 = "sandbox.wis-tns.org"
url_tns_api         = "https://" + TNS + "/api/get"

TNS_BOT_ID          = "YOUR_BOT_ID_HERE"
TNS_BOT_NAME        = "YOUR_BOT_NAME_HERE"
TNS_API_KEY         = "YOUR_BOT_API_KEY_HERE"

# list that represents json file for search obj
search_obj          = [("ra", ""), ("dec", ""), ("radius", ""), ("units", ""), ("objname", ""), 
                       ("objname_exact_match", 0), ("internal_name", ""), 
                       ("internal_name_exact_match", 0), ("objid", ""), ("public_timestamp", "")]

# list that represents json file for get obj
get_obj             = [("objname", ""), ("objid", ""), ("photometry", "0"), ("spectra", "1")]

# file url (for downloading file from TNS)
file_tns_url        = "Here put url of a file you want to download from TNS." 

# current working directory
cwd                 = os.getcwd()
# directory for downloaded files
#download_dir        = os.path.join(cwd, "downloaded_files")
download_dir        = os.getcwd()
# external http errors
ext_http_errors     = [403, 500, 503]
err_msg             = ["Forbidden", "Internal Server Error: Something is broken", "Service Unavailable"]

#----------------------------------------------------------------------------------

def set_bot_tns_marker():
    tns_marker = 'tns_marker{"tns_id": "' + str(TNS_BOT_ID) + '", "type": "bot", "name": "' + TNS_BOT_NAME + '"}'
    return tns_marker

def format_to_json(source):
    parsed = json.loads(source, object_pairs_hook = OrderedDict)
    result = json.dumps(parsed, indent = 4)
    return result

def is_string_json(string):
    try:
        json_object = json.loads(string)
    except Exception:
        return False
    return json_object

def print_status_code(response):
    json_string = is_string_json(response.text)
    if json_string != False:
        print ("status code ---> [ " + str(json_string['id_code']) + " - '" + json_string['id_message'] + "' ]\n")
    else:
        status_code = response.status_code
        if status_code == 200:
            status_msg = 'OK'
        elif status_code in ext_http_errors:
            status_msg = err_msg[ext_http_errors.index(status_code)]
        else:
            status_msg = 'Undocumented error'
        print ("status code ---> [ " + str(status_code) + " - '" + status_msg + "' ]\n")

def search():
    search_url = url_tns_api + "/search"
    tns_marker = set_bot_tns_marker()
    headers = {'User-Agent': tns_marker}
    json_file = OrderedDict(search_obj)
    search_data = {'api_key': TNS_API_KEY, 'data': json.dumps(json_file)}
    response = requests.post(search_url, headers = headers, data = search_data)
    return response

def get():
    get_url = url_tns_api + "/object"
    tns_marker = set_bot_tns_marker()
    headers = {'User-Agent': tns_marker}
    json_file = OrderedDict(get_obj)
    get_data = {'api_key': TNS_API_KEY, 'data': json.dumps(json_file)}
    response = requests.post(get_url, headers = headers, data = get_data)
    return response

def get_file():
    filename = os.path.basename(file_tns_url)
    tns_marker = set_bot_tns_marker()
    headers = {'User-Agent': tns_marker}
    api_data = {'api_key': TNS_API_KEY}
    print ("Downloading file '" + filename + "' from the TNS...\n")
    response = requests.post(file_tns_url, headers = headers, data = api_data, stream = True)    
    print_status_code(response)
    path = os.path.join(download_dir, filename)
    if response.status_code == 200:
        with open(path, 'wb') as f:
            for chunk in response:
                f.write(chunk)
        print ("File was successfully downloaded.\n")
    else:
        print ("File was not downloaded.\n")

def print_response(response, json_file, counter):
    response_code = str(response.status_code) if json_file == False else str(json_file['id_code'])
    stats = 'Test #' + str(counter) + '| return code: ' + response_code + \
            ' | Total Rate-Limit: ' + str(response.headers.get('x-rate-limit-limit')) + \
            ' | Remaining: ' + str(response.headers.get('x-rate-limit-remaining')) + \
            ' | Reset: ' + str(response.headers.get('x-rate-limit-reset'))
    if(response.headers.get('x-cone-rate-limit-limit') != None):
        stats += ' || Cone Rate-Limit: ' + str(response.headers.get('x-cone-rate-limit-limit')) + \
                 ' | Cone Remaining: ' + str(response.headers.get('x-cone-rate-limit-remaining')) + \
                 ' | Cone Reset: ' + str(response.headers.get('x-cone-rate-limit-reset'))
    print (stats)

def get_reset_time(response):
    # If any of the '...-remaining' values is zero, return the reset time
    for name in response.headers:
        value = response.headers.get(name)
        if name.endswith('-remaining') and value == '0':
            return int(response.headers.get(name.replace('remaining', 'reset')))
    return None

def rate_limit_handling():
    counter = 0
    while True:
        counter = counter + 1
        response = search()
        json_file = is_string_json(response.text)
        print_response(response, json_file, counter)
        # Checking if rate-limit reached (...-remaining = 0)
        reset = get_reset_time(response)
        # A general verification if not some error 
        if (response.status_code == 200):
            if reset != None:
                # Sleeping for reset + 1 sec
                print("Sleep for " + str(reset + 1) + " sec") 
                time.sleep(reset + 1)
        	    # Can continue to submit requests...
                print ("Continue to submit requests...")
                for i in range(3):
                    counter = counter + 1
                    response = search()
                    json_file = is_string_json(response.text)
                    print_response(response, json_file, counter)
                print ("etc...\n") 
                break
        else:
            print_status_code(response)       
            break

#----------------------------------------------------------------------------------


#ztf_object_id="ZTF19aapreis"

#locus = antares_client.search.get_by_ztf_object_id(ztf_object_id)
print(locus.properties['ztf_object_id'],locus.ra,locus.dec)


# EXAMPLE 1 (search obj)
search_obj          = [("ra", locus.ra), ("dec", locus.dec), ("radius", "5"), ("units", "arcsec"), 
                       ("objname", ""), ("objname_exact_match", 0), ("internal_name", ""), 
                       ("internal_name_exact_match", 0), ("objid", ""), ("public_timestamp", "")]
response = search()
json_data = format_to_json(response.text)
print (json_data)

"""
# EXAMPLE 2 (get obj)
get_obj             = [("objname", "2017A"), ("objid", ""), ("photometry", "1"), ("spectra", "1")]
response = get()
json_data = format_to_json(response.text)
print (json_data)
"""

"""
# EXAMPLE 3 (get file from TNS)
file_tns_url        = "https://" + TNS + "/system/files/uploaded/"\
                      "Padova-Asiago/tns_2017A_2457777.69_Ekar_AFOSC_Padova-Asiago.txt"
get_file()
"""

"""
# EXAMPLE 4 (test rate-limit search)
search_obj          = [("ra", ""), ("dec", ""), ("radius", ""), ("units", ""), 
                       ("objname", "2021rak"), ("objname_exact_match", 0), ("internal_name", ""), 
                       ("internal_name_exact_match", 0), ("objid", ""), ("public_timestamp", "")]
rate_limit_handling()
"""

"""
# EXAMPLE 5 (test rate-limit cone search)
search_obj          = [("ra", "15:57:28"), ("dec", "+30:03:39"), ("radius", "5"), ("units", "arcsec"), 
                       ("objname", ""), ("objname_exact_match", 0), ("internal_name", ""), 
                       ("internal_name_exact_match", 0), ("objid", ""), ("public_timestamp", "")]
rate_limit_handling()
"""

#----------------------------------------------------------------------------------







                                        	   


ZTF19aapreis 314.2623563407407 14.204495246296297
{
    "id_code": 200,
    "id_message": "OK",
    "data": {
        "received_data": {
            "ra": 314.2623563407407,
            "dec": 14.204495246296297,
            "radius": 5,
            "units": "arcsec",
            "objname": "",
            "objname_exact_match": 0,
            "internal_name": "",
            "internal_name_exact_match": 0,
            "objid": "",
            "public_timestamp": ""
        },
        "reply": [
            {
                "objname": "2019dsg",
                "prefix": "AT",
                "objid": 36507
            }
        ]
    }
}


'\n# EXAMPLE 5 (test rate-limit cone search)\nsearch_obj          = [("ra", "15:57:28"), ("dec", "+30:03:39"), ("radius", "5"), ("units", "arcsec"), \n                       ("objname", ""), ("objname_exact_match", 0), ("internal_name", ""), \n                       ("internal_name_exact_match", 0), ("objid", ""), ("public_timestamp", "")]\nrate_limit_handling()\n'

In [17]:
#we can print out the result of the TNS query. Note it provides the url of spectral file in "asciifile".
a=response.json()
#print(a)
#a['data']['reply'][0]['objname']

get_obj             = [("objname", a['data']['reply'][0]['objname']), ("objid", ""), ("photometry", "0"), ("spectra", "1")]
response = get()
json_data = format_to_json(response.text)
print (json_data)

{
    "id_code": 200,
    "id_message": "OK",
    "data": {
        "received_data": {
            "objname": "2019dsg",
            "objid": "",
            "photometry": 0,
            "spectra": 1
        },
        "reply": {
            "objname": "2019dsg",
            "name_prefix": "AT",
            "objid": 36507,
            "object_type": {
                "name": "TDE",
                "id": 120
            },
            "redshift": 0.0512,
            "ra": "20:57:02.974",
            "dec": "+14:12:15.86",
            "radeg": 314.2623926,
            "decdeg": 14.2044063,
            "radeg_err": 0.00027777777777778,
            "decdeg_err": 0,
            "hostname": null,
            "host_redshift": null,
            "internal_names": "ZTF19aapreis, ATLAS19klx",
            "discoverer_internal_name": "ZTF19aapreis",
            "discoverydate": "2019-04-09 11:09:28.000",
            "discoverer": "J. Nordin, V. Brinnel, M. Giomi, J. van Santen (HU Berlin), A. Gal-Y

In [18]:
#get the spectral file url from the query
b=response.json()
#print(b)
b['data']['reply']['spectra'][0]['asciifile']

'https://sandbox.wis-tns.org/system/files/uploaded/ePESSTO%2B/tns_2019dsg_2019-05-13_08-40-46_ESO-NTT_EFOSC2-NTT_ePESSTO%2B.asci'

In [19]:
#download the spectral file
file_tns_url=b['data']['reply']['spectra'][0]['asciifile']
get_file()


status code ---> [ 200 - 'OK' ]

File was successfully downloaded.



In [20]:
#get the local filename, read in file, reformat into TOM compatible CSV format
filename = os.path.basename(file_tns_url)
#print(filename)
spec=pd.read_table(filename)
spec.columns=['wavelength','flux']
spec.to_csv(filename+'.csv',index=False)
spec #print out table

,wavelength,flux
0,3649.565780,6.672010e-16
1,3655.083709,6.667040e-16
2,3660.601637,6.899820e-16
3,3666.119566,6.897360e-16
4,3671.637495,6.912930e-16
...,...,...
1009,9217.155738,1.842900e-16
1010,9222.673666,1.767300e-16
1011,9228.191595,1.870900e-16
1012,9233.709524,1.978910e-16


In [21]:
#upload the spectral file to ANTARES TOM
m = MultipartEncoder(
    fields={'target': str(target_id), 'data_product_type': 'spectroscopy',
            'file': (filename+'.csv', open(filename+'.csv', 'rb'), 'text/csv')}
    )

#print(m)
r = requests.post('https://tom.antares.noirlab.edu/api/dataproducts/', data=m,
                  headers={'Content-Type': m.content_type, 'Authorization': f'token {token}'})

print(r.content)

b'{"id":29,"product_id":null,"target":31,"observation_record":null,"data":"http://tom.antares.noirlab.edu/data/ZTF19aapreis/none/tns_2019dsg_2019-05-13_08-40-46_ESO-NTT_EFOSC2-NTT_ePESSTO2B.asci.csv","extra_data":"","data_product_type":"spectroscopy","reduceddatum_set":[],"groups":[]}'


## Glossary 1: how to enable token authentication and retrieve token from TOMtoolkit

The examples shown in this notebook require token authentication with TOMtoolkit, which can be enabled as follows:

1. In the "urls.py", please add the following lines:

```python
from rest_framework.authtoken.views import obtain_auth_token


urlpatterns = [
    path('', include('tom_common.urls')),
    path('api-token-auth/', obtain_auth_token, name='api_token_auth'),
]
```

2. In the "settings.py", we need to include restframe token-authentication related packages as following:

- in INSTALLED_APP:

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    ...
    'rest_framework',
    'rest_framework.authtoken',
    ...
]
```

- in REST_FRAMEWORK:

```python
REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
    ],
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework.authentication.TokenAuthentication',
    ],
    'TEST_REQUEST_DEFAULT_FORMAT': 'json',
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.LimitOffsetPagination',
    'PAGE_SIZE': 100
}
```

3. Once the token authentication is enabled, users can generate their respective API tokens using the <a href="https://httpie.io">HTTPie command</a>:

http post http://127.0.0.1:8000/api-token-auth/ username=YOUR_USERNAME password=YOUR_PASSWORD




## Glossary 2: using API to query dataproduct (in json format)

In [6]:
#get dataproduct associated with targets
response = api('GET', 'https://tom.antares.noirlab.edu/api/dataproducts')

#comment out printing as the data product is rather long
print(f'HTTP code: {response.status_code}, {response.reason}')
if response.status_code in (200, 400):
    print(f'JSON response: {response.json()}')




HTTP code: 200, OK
JSON response: {'count': 1, 'next': None, 'previous': None, 'results': [{'id': 21, 'product_id': None, 'target': 27, 'observation_record': None, 'data': 'http://tom.antares.noirlab.edu/data/ZTF19aapreis/none/ZTF19aapreis_8V0B2jU.csv', 'extra_data': '', 'data_product_type': 'photometry', 'reduceddatum_set': [{'data_product': 21, 'data_type': 'photometry', 'source_name': '', 'source_location': '', 'timestamp': '2019-04-09T11:09:27.999347Z', 'value': {'magnitude': 18.87879943847656, 'filter': 'R', 'error': 0.14493699371814728}}, {'data_product': 21, 'data_type': 'photometry', 'source_name': '', 'source_location': '', 'timestamp': '2019-04-20T10:42:10.995833Z', 'value': {'magnitude': 18.299699783325195, 'filter': 'R', 'error': 0.09438800066709517}}, {'data_product': 21, 'data_type': 'photometry', 'source_name': '', 'source_location': '', 'timestamp': '2019-04-28T11:45:17.000645Z', 'value': {'magnitude': 18.024599075317386, 'filter': 'R', 'error': 0.06977300345897675}}, {

In [26]:
#in a shorter format:
data_list=response.json()
data_list_id=[]
for i in data_list['results']:
    print(i['id'],i['target'],i['data_product_type'])
    data_list_id.append(i['id'])

20 26 photometry


In [27]:
phot=data_list['results'][0]
#phot=data_list['results'][0]['reduceddatum_set']
phot

{'id': 20,
 'product_id': None,
 'target': 26,
 'observation_record': None,
 'data': 'http://tom.antares.noirlab.edu/data/ZTF19aapreis/none/ZTF19aapreis_2ZgTNHB.csv',
 'extra_data': '',
 'data_product_type': 'photometry',
 'reduceddatum_set': [{'data_product': 20,
   'data_type': 'photometry',
   'source_name': '',
   'source_location': '',
   'timestamp': '2019-04-09T11:09:27.999347Z',
   'value': {'magnitude': 18.87879943847656,
    'filter': 'R',
    'error': 0.14493699371814728}},
  {'data_product': 20,
   'data_type': 'photometry',
   'source_name': '',
   'source_location': '',
   'timestamp': '2019-04-20T10:42:10.995833Z',
   'value': {'magnitude': 18.299699783325195,
    'filter': 'R',
    'error': 0.09438800066709517}},
  {'data_product': 20,
   'data_type': 'photometry',
   'source_name': '',
   'source_location': '',
   'timestamp': '2019-04-28T11:45:17.000645Z',
   'value': {'magnitude': 18.024599075317386,
    'filter': 'R',
    'error': 0.06977300345897675}},
  {'data_pro

In [18]:
spec=data_list['results'][1]
spec


{'id': 4,
 'product_id': None,
 'target': 1,
 'observation_record': None,
 'data': 'http://tom.antares.noirlab.edu/data/ZTF19aapreis/none/tns_2019dsg_2019-05-13_08-40-46_ESO-NTT_EFOSC2-NTT_ePESSTO.fits',
 'extra_data': '',
 'data_product_type': 'spectroscopy',
 'reduceddatum_set': [{'data_product': 4,
   'data_type': 'spectroscopy',
   'source_name': '',
   'source_location': '',
   'timestamp': '2021-07-27T18:02:52.305378Z',
   'value': {'flux': [6.524424315746734e-16,
     6.672007569339101e-16,
     6.667044687612175e-16,
     6.899817976992446e-16,
     6.897364304032847e-16,
     6.912934525035736e-16,
     6.863205995748053e-16,
     6.416827474806932e-16,
     6.611418003108334e-16,
     6.66259424992928e-16,
     6.516721877868156e-16,
     6.663569913096857e-16,
     6.480401459614332e-16,
     6.713034016209112e-16,
     6.704740879284704e-16,
     6.588184471727766e-16,
     6.535109910024495e-16,
     6.367567213352587e-16,
     6.296575411681214e-16,
     6.632861135778413